https://crypto.stackexchange.com/questions/2249/how-does-one-attack-a-two-time-pad-i-e-one-time-pad-with-key-reuse

#### WARNING: DO NOT DO INVENT NOR IMPLEMENT YOUR OWN CRYPTO
This is for demonstration purposes and OTP attack game only!

In [1]:
import secrets

In [2]:
def gen_key(length):
    """Vrne nakljucni niz bajtov podane dolzine"""
    return secrets.token_bytes(length)

In [3]:
def xor_bytes(s1, s2):
    """Izvede operacijo XOR med podanima seznamoma bajtov in vrne seznam bajtov"""
    return bytes(a ^ b for a, b in zip(s1, s2))

In [4]:
def enc_otp(key, pt):
    """Sifrirni algoritem enkratne podloge"""
    return xor_bytes(key, pt)

In [5]:
def dec_otp(key, ct):
    """Desifrirni algoritem enkratne podloge"""
    return xor_bytes(key, ct)

In [35]:
def prepare_pt():
    def normalize(msg):
        msg = msg.lower()
        msg = msg.replace("č", "c")
        msg = msg.replace("š", "s")
        msg = msg.replace("ž", "z")
        msg = " ".join(msg.split())
        return "".join(x for x in msg if x in 'abcdefghijklmnoprsštuvz ')

    data = [
    "Slovenija je evropska država z zemljepisno lego na skrajnem severu Sredozemlja in na skrajnem jugu Srednje Evrope",
    "Skozi slovensko zgodovino so pomembni kulturni vplivi prihajali iz srednjeevropskega in apeninskega kulturnega prostora.",
    "Slovenija ima gospodarsko ureditev, ki temelji na prostem trgu, zdaj je po mednarodnih merilih uvrščena med države z zelo visokim indeksom človekovega razvoja",
    "Poimenovanje za Slovence izvira iz besede Slovani izvor te pa je nejasen je izpeljanka iz korena slov ki je med drugim ohranjen v nekaterih vodnih imenih širom vzhodne Evrope",
    "Prvič je bilo ime uporabljeno za državno tvorbo Federalna Slovenija, ki je po sklepu zasedanja Slovenskega narodnoosvobodilnega sveta pristopila v Demokratično federativno Jugoslavijo",
    "Slovanski predniki današnjih Slovencev so se na ilirsko ozemlje Slovenije naselili v sestem stoletju. V sedmem stoletju se je oblikovala Karantanija, prva država alpskih Slovanov.",
    "Okoli leta tisoc so bili napisani Brižinski spomeniki, prvi pisni dokument v slovenščini in prvi slovanski zapis v latinici.",
    "V Pomladi narodov so med številnimi narodi tudi Slovenci s političnim programom zahtevali Zedinjeno Slovenijo",
    "Geografsko leži Slovenija v srednji Evropi oziroma v jugovzhodni Evropi na stičišču Alp, Dinarskega gorstva, Panonske nižine in Sredozemlja.",
    "Največ slovenskega površja sestavljajo kamnine iz časa, mezozoika (apnenec, dolomit) in kenozoika, torej terciarja (lapor, peščenjak, fliš) in kvartarja (prod, pesek, glina, konglomerat).",
    ]
    # target = "ce sifriras z enkratno podlogo, ne uporabi istega kljuce vec kot enkrat"
    length = len(min(data, key=lambda x:len(x)))
    return [normalize(s)[:length] for s in data]

In [36]:
plain_texts = prepare_pt()
secret_key = gen_key(len(max(plain_texts, key=lambda x:len(x))))
cipher_texts = [enc_otp(secret_key, pt.encode("utf-8")) for pt in plain_texts]

### Nekaj namigov
- Kodiranje je ASCII. V abecedi ni šumnikov, so le (velike tiskane) črke in presledek
- Pri črki bo bit 6 vedno nastavljen. To pomeni, da je vsaka črka zakodirana z znakom, ki je > 64
- Pri presledku je bit 6 vedno nenastavljen. Presledek in ASCII vrednost 32
- Ko naredimo XOR črke in presledka bo bit 6 nastavljen. Torej če je ASCII vrednost v $d$ nad 64, to pomeni, da gledamo vrednost, ki je nastala z XOR presledka in neke črke: Če XOR-amo 32 (presledek) dobimo črko v enem čistopisu (sicer ne vemo katerem)


| $\oplus$       | A-Z (65-90) | presledek (32) |
| ---------------|-------------|----------------|
| A-Z (65-90)    |    $\leq 32$  |    $\geq 65$     |
| Presledek (32) |    $\geq 65$  |       $0$      |

In [54]:
def guess_key_old(ciphertexts):
    key = bytearray(len(ciphertexts[0]))

    for k in range(len(ciphertexts[0])):
        guess = None

        for c1 in ciphertexts:
            for c2 in ciphertexts:
                if c1 == c2:
                    continue

                xor = c1[k] ^ c2[k]
                
                if 0 < xor < 65:
                    guess = ord('?')
                    break

                guess = ' '
            
            if guess == ' ': 
                key[k] = c1[k] ^ ord(' ')
                break
        
    return key

def multiple_xor(position, ciphertexts):
    for c1 in ciphertexts:
        if all(c1[position] ^ c2[position] >= 65 for c2 in ciphertexts if c2 != c1):
            return c1[position] ^ ord(' ')
    
    return 0

def guess_key(ciphertexts):
    key = bytearray(len(ciphertexts[0]))

    for k in range(len(ciphertexts[0])):
        key[k] = multiple_xor(k, ciphertexts)
    return key
    
gk = guess_key(cipher_texts)
print(sum(1 for a, g in zip(secret_key, gk) if a == g) / len(secret_key))
# gk

0.3944954128440367


In [ ]:
def display(bytez):
    return "".join(chr(i) for i in bytez)

def attempts(crib):
    attempts = []
    binary = [ord(c) for c in crib]

    for i in range(len(xor) - len(binary)):
        guess = [x ^ y for x, y in zip(binary, xor[i:i+len(binary)])]
        string = display(guess)
        attempts.append(( entropy(string), string ))

    attempts.sort(key=lambda tup: tup[0])
    return attempts[0][1]

def entropy1(string):
    e = 0
    string = string.replace(" ", "")#.lower()
    for i in range(len(string)):
        for j in range(i+1, len(string)):
            e += (ord(string[i]) - ord(string[j]))**2
    
    return e / len(string) # / len(string)

def entropy(string):
    e = 0
    string = string.replace(" ", "").upper()
    for i in range(len(string)):
        e += (77.5 - ord(string[i]))**2 # 70.55
    
    return e / len(string)

In [ ]:
# Electronic mail (email or e-mail) is a method of exchanging messages (mail) between people using Internet Mail Extensions (MIME) to carry text in other character sets and multimedia content 

In [ ]:
attempts("in other")

In [ ]:
b = attempts(
    " electronic devices ")
b

In [ ]:
cts